https://medium.com/swlh/web-scraping-python-requests-and-beautifulsoup-45d5f48f5a1

https://sparkbyexamples.com/spark/spark-read-text-file-rdd-dataframe/

https://medium.com/swlh/text2sql-in-spark-nlp-converting-natural-language-questions-to-sql-queries-on-scale-6ae9a9061d74

https://www.oreilly.com/library/view/learning-spark-2nd/9781492050032/ch04.html

In [1]:
!python -m pip install html2text

In [2]:
!python -m pip install mistletoe

https://github.com/webrecorder/warcio

In [3]:
!pip install warcio

In [4]:
!python -m pip install cdx_toolkit

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import json
import sys
import os.path
import re
import logging
import pandas as pd
import requests
import warcio
import mistletoe
import cdx_toolkit
import zlib

from warcio import ArchiveIterator
from contextlib import closing
from html2text import HTML2Text
from bs4 import BeautifulSoup
from pathlib import Path
from IPython.display import HTML as HTML_raw, display

In [7]:
websites = pd.read_csv("./ccmain/ccmain_2021-04_Long2.txt", header = None, sep = '|')

In [8]:
websites.head()

,0,1,2,3
0,NaN,https://globalannal.wordpress.com/tag/careers/...,crawl-data/CC-MAIN-2021-04/segments/1610703528...,NaN
1,NaN,https://globalannal.wordpress.com/tag/jobs-num...,crawl-data/CC-MAIN-2021-04/segments/1610703528...,NaN
2,NaN,https://globalannal.wordpress.com/tag/jobs-rep...,crawl-data/CC-MAIN-2021-04/segments/1610703528...,NaN
3,NaN,https://globalshapersbucharest.wordpress.com/2...,crawl-data/CC-MAIN-2021-04/segments/1610703506...,NaN
4,NaN,https://globalyouthbook.wordpress.com/tag/care...,crawl-data/CC-MAIN-2021-04/segments/1610704798...,NaN


In [9]:
websites.columns

Int64Index([0, 1, 2, 3], dtype='int64')

In [10]:
websites.drop([0, 3], axis=1, inplace=True)
websites.head()

,1,2
0,https://globalannal.wordpress.com/tag/careers/...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
1,https://globalannal.wordpress.com/tag/jobs-num...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
2,https://globalannal.wordpress.com/tag/jobs-rep...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
3,https://globalshapersbucharest.wordpress.com/2...,crawl-data/CC-MAIN-2021-04/segments/1610703506...
4,https://globalyouthbook.wordpress.com/tag/care...,crawl-data/CC-MAIN-2021-04/segments/1610704798...


In [11]:
websites.columns = ['url_link', 'warc_file']
websites.head()

,url_link,warc_file
0,https://globalannal.wordpress.com/tag/careers/...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
1,https://globalannal.wordpress.com/tag/jobs-num...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
2,https://globalannal.wordpress.com/tag/jobs-rep...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
3,https://globalshapersbucharest.wordpress.com/2...,crawl-data/CC-MAIN-2021-04/segments/1610703506...
4,https://globalyouthbook.wordpress.com/tag/care...,crawl-data/CC-MAIN-2021-04/segments/1610704798...


In [12]:
websites.url_link.value_counts()

https://seeker.worksourcewa.com/jobsearch/powersearch.aspx?where=Washington                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      7
https://www.xn--itbaw-ova.de/jobs/151816/werkstudent-w-strich-m-strich-d-fuer-bachelor-master-oder-projektarbeiten-softwareentwicklung-java-in-stuttgart-und-karlsruhe                                                               

In [13]:
websites.url_link.value_counts().keys()

Index(['https://seeker.worksourcewa.com/jobsearch/powersearch.aspx?where=Washington                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  ',
       'https://www.xn--itbaw-ova.de/jobs/151816/werkstudent-w-strich-m-strich-d-fuer-bachelor-master-oder-projektarbeiten-softwareentwicklung-java-in-stuttgart-und-karlsruhe                                                  

In [14]:
urlList = []
for i in websites.url_link.value_counts().keys():
    urlList.append(i.replace(' ',''))
    
print(urlList[:10])

['https://seeker.worksourcewa.com/jobsearch/powersearch.aspx?where=Washington', 'https://www.xn--itbaw-ova.de/jobs/151816/werkstudent-w-strich-m-strich-d-fuer-bachelor-master-oder-projektarbeiten-softwareentwicklung-java-in-stuttgart-und-karlsruhe', 'https://www.xn--itbaw-ova.de/jobs/151789/werkstudent-w-strich-m-strich-d-softwareentwicklung-java-in-stuttgart-und-karlsruhe', 'https://www.xn--itbaw-ova.de/jobs/152230/senior-software-engineer-w-strich-m-strich-d-sap-in-stuttgart-karlsruhe-und-reutlingen', 'https://www.xn--itbaw-ova.de/jobs/152535/consultant-w-strich-m-strich-d-microsoft-business-intelligence-in-stuttgart-und-karlsruhe', 'https://www.xn--itbaw-ova.de/jobs/152501/consultant-w-strich-m-strich-d-sap-abap-in-stuttgart-karlsruhe-und-reutlingen', 'https://www.xn--itbaw-ova.de/jobs/152418/ios-developer-w-strich-m-strich-d-fuer-die-adesso-mobile-solutions-in-stuttgart-und-karlsruhe', 'http://www.work521.com/jobs/2452.html', 'https://www.vdk.de/bayern/pages/presse/vdk-zeitung/vdk-

In [15]:
statesList = ["United States of America","USA","AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS",
              "KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA",
              "RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY","DC","GU","MH","MP","PR","VI"]
itList = ["engineer","Engineer","scientist","Scientist","developer","Developer","Analyst","analyst","Operator","operator"]
qaList = ["Qualification","Qualifications","qualification","qualifications","Certification","Certifications","certification",
          "certifications"]

In [18]:
USAJobsList = []
fout = open("./ads/2021JAN_USAjobAD.txt","w")
for i in range(0, len(urlList)):
    filename = "./dat/100k_jobADs/"+str(i+1)+"_2021JAN_jobAD.txt"
    if (os.path.exists(filename)):
        with open(filename, encoding='utf-8') as fin:
            #all_USAs = re.findall('USA', fin.read())
            all_USAs = re.findall(r"(?=("+'|'.join(statesList)+r"))", fin.read())
            nUSAs = len(all_USAs)
            if nUSAs > 0:
                print(str(i+1)+"_2021JAN_jobAD.txt",' ',nUSAs)
                fout.write(str(i+1)+'_2021JAN_jobAD.txt'+' '+str(nUSAs)+'\n')
                USAJobsList.append(i+1)
            else:
                continue
    else:
        continue
            
fout.close()
print(USAJobsList)

1_2021JAN_jobAD.txt   49
11_2021JAN_jobAD.txt   3
21_2021JAN_jobAD.txt   4
22_2021JAN_jobAD.txt   3
23_2021JAN_jobAD.txt   3
27_2021JAN_jobAD.txt   4
28_2021JAN_jobAD.txt   4
39_2021JAN_jobAD.txt   4
41_2021JAN_jobAD.txt   4
45_2021JAN_jobAD.txt   3
49_2021JAN_jobAD.txt   1
53_2021JAN_jobAD.txt   24
54_2021JAN_jobAD.txt   45
55_2021JAN_jobAD.txt   17
56_2021JAN_jobAD.txt   8
62_2021JAN_jobAD.txt   15
65_2021JAN_jobAD.txt   3
74_2021JAN_jobAD.txt   3
84_2021JAN_jobAD.txt   6
87_2021JAN_jobAD.txt   17
91_2021JAN_jobAD.txt   4
96_2021JAN_jobAD.txt   5
108_2021JAN_jobAD.txt   3
115_2021JAN_jobAD.txt   1
116_2021JAN_jobAD.txt   1
117_2021JAN_jobAD.txt   1
121_2021JAN_jobAD.txt   4
124_2021JAN_jobAD.txt   18
125_2021JAN_jobAD.txt   3
129_2021JAN_jobAD.txt   6
132_2021JAN_jobAD.txt   2
134_2021JAN_jobAD.txt   6
135_2021JAN_jobAD.txt   4
147_2021JAN_jobAD.txt   5
153_2021JAN_jobAD.txt   5
156_2021JAN_jobAD.txt   1
168_2021JAN_jobAD.txt   24
169_2021JAN_jobAD.txt   3
177_2021JAN_jobAD.txt   3
1

1085_2021JAN_jobAD.txt   10
1086_2021JAN_jobAD.txt   14
1087_2021JAN_jobAD.txt   4
1091_2021JAN_jobAD.txt   6
1098_2021JAN_jobAD.txt   5
1099_2021JAN_jobAD.txt   6
1100_2021JAN_jobAD.txt   3
1102_2021JAN_jobAD.txt   4
1105_2021JAN_jobAD.txt   2
1106_2021JAN_jobAD.txt   11
1108_2021JAN_jobAD.txt   4
1110_2021JAN_jobAD.txt   8
1111_2021JAN_jobAD.txt   5
1117_2021JAN_jobAD.txt   5
1118_2021JAN_jobAD.txt   8
1120_2021JAN_jobAD.txt   118
1123_2021JAN_jobAD.txt   2
1124_2021JAN_jobAD.txt   5
1127_2021JAN_jobAD.txt   4
1135_2021JAN_jobAD.txt   200
1137_2021JAN_jobAD.txt   5
1139_2021JAN_jobAD.txt   11
1143_2021JAN_jobAD.txt   3
1144_2021JAN_jobAD.txt   22
1146_2021JAN_jobAD.txt   14
1147_2021JAN_jobAD.txt   620
1148_2021JAN_jobAD.txt   1
1149_2021JAN_jobAD.txt   187
1150_2021JAN_jobAD.txt   3
1152_2021JAN_jobAD.txt   52
1158_2021JAN_jobAD.txt   20
1160_2021JAN_jobAD.txt   14
1161_2021JAN_jobAD.txt   9
1163_2021JAN_jobAD.txt   14
1169_2021JAN_jobAD.txt   3
1173_2021JAN_jobAD.txt   12
1174_2021

2037_2021JAN_jobAD.txt   177
2042_2021JAN_jobAD.txt   10
2043_2021JAN_jobAD.txt   14
2045_2021JAN_jobAD.txt   2
2059_2021JAN_jobAD.txt   4
2069_2021JAN_jobAD.txt   14
2071_2021JAN_jobAD.txt   20
2073_2021JAN_jobAD.txt   4
2074_2021JAN_jobAD.txt   11
2077_2021JAN_jobAD.txt   3
2078_2021JAN_jobAD.txt   11
2084_2021JAN_jobAD.txt   45
2085_2021JAN_jobAD.txt   3
2087_2021JAN_jobAD.txt   1
2088_2021JAN_jobAD.txt   16
2090_2021JAN_jobAD.txt   3
2092_2021JAN_jobAD.txt   8
2093_2021JAN_jobAD.txt   3
2094_2021JAN_jobAD.txt   10
2095_2021JAN_jobAD.txt   11
2101_2021JAN_jobAD.txt   4
2104_2021JAN_jobAD.txt   11
2112_2021JAN_jobAD.txt   82
2116_2021JAN_jobAD.txt   2
2120_2021JAN_jobAD.txt   54
2121_2021JAN_jobAD.txt   3
2125_2021JAN_jobAD.txt   1
2129_2021JAN_jobAD.txt   3
2130_2021JAN_jobAD.txt   8
2132_2021JAN_jobAD.txt   14
2135_2021JAN_jobAD.txt   13
2151_2021JAN_jobAD.txt   25
2156_2021JAN_jobAD.txt   4
2161_2021JAN_jobAD.txt   5
2166_2021JAN_jobAD.txt   8
2167_2021JAN_jobAD.txt   8
2169_2021J

2964_2021JAN_jobAD.txt   1095
2970_2021JAN_jobAD.txt   11
2972_2021JAN_jobAD.txt   7
2975_2021JAN_jobAD.txt   10
2978_2021JAN_jobAD.txt   2
2982_2021JAN_jobAD.txt   108
2983_2021JAN_jobAD.txt   16
2984_2021JAN_jobAD.txt   10
2988_2021JAN_jobAD.txt   10
2992_2021JAN_jobAD.txt   5
2996_2021JAN_jobAD.txt   10
3000_2021JAN_jobAD.txt   4
3009_2021JAN_jobAD.txt   7
3010_2021JAN_jobAD.txt   24
3011_2021JAN_jobAD.txt   1
3015_2021JAN_jobAD.txt   89
3018_2021JAN_jobAD.txt   3
3019_2021JAN_jobAD.txt   5
3020_2021JAN_jobAD.txt   11
3024_2021JAN_jobAD.txt   6
3026_2021JAN_jobAD.txt   177
3035_2021JAN_jobAD.txt   6
3040_2021JAN_jobAD.txt   16
3043_2021JAN_jobAD.txt   1
3047_2021JAN_jobAD.txt   3
3048_2021JAN_jobAD.txt   27
3050_2021JAN_jobAD.txt   3
3052_2021JAN_jobAD.txt   14
3053_2021JAN_jobAD.txt   1
3059_2021JAN_jobAD.txt   14
3062_2021JAN_jobAD.txt   5
3063_2021JAN_jobAD.txt   13
3070_2021JAN_jobAD.txt   45
3072_2021JAN_jobAD.txt   15
3074_2021JAN_jobAD.txt   1
3090_2021JAN_jobAD.txt   13
3093

3851_2021JAN_jobAD.txt   3
3854_2021JAN_jobAD.txt   3
3859_2021JAN_jobAD.txt   4
3861_2021JAN_jobAD.txt   13
3865_2021JAN_jobAD.txt   4
3868_2021JAN_jobAD.txt   4
3872_2021JAN_jobAD.txt   9
3874_2021JAN_jobAD.txt   12
3877_2021JAN_jobAD.txt   6
3880_2021JAN_jobAD.txt   1
3885_2021JAN_jobAD.txt   4
3887_2021JAN_jobAD.txt   11
3891_2021JAN_jobAD.txt   4
3894_2021JAN_jobAD.txt   4
3900_2021JAN_jobAD.txt   5
3907_2021JAN_jobAD.txt   39
3916_2021JAN_jobAD.txt   11
3917_2021JAN_jobAD.txt   2
3919_2021JAN_jobAD.txt   5
3921_2021JAN_jobAD.txt   14
3925_2021JAN_jobAD.txt   3
3926_2021JAN_jobAD.txt   16
3929_2021JAN_jobAD.txt   34
3930_2021JAN_jobAD.txt   4
3931_2021JAN_jobAD.txt   11
3932_2021JAN_jobAD.txt   2
3933_2021JAN_jobAD.txt   10
3942_2021JAN_jobAD.txt   4
3943_2021JAN_jobAD.txt   2
3947_2021JAN_jobAD.txt   14
3949_2021JAN_jobAD.txt   5
3950_2021JAN_jobAD.txt   3
3951_2021JAN_jobAD.txt   3
3952_2021JAN_jobAD.txt   18
3953_2021JAN_jobAD.txt   24
3954_2021JAN_jobAD.txt   68
3956_2021JAN_j

...

95462_2021JAN_jobAD.txt   2
95465_2021JAN_jobAD.txt   11
95467_2021JAN_jobAD.txt   27
95468_2021JAN_jobAD.txt   3
95472_2021JAN_jobAD.txt   7
95473_2021JAN_jobAD.txt   4
95475_2021JAN_jobAD.txt   27
95478_2021JAN_jobAD.txt   1
95480_2021JAN_jobAD.txt   14
95485_2021JAN_jobAD.txt   5
95486_2021JAN_jobAD.txt   14
95490_2021JAN_jobAD.txt   6
95493_2021JAN_jobAD.txt   18
95497_2021JAN_jobAD.txt   2
95498_2021JAN_jobAD.txt   8
95500_2021JAN_jobAD.txt   3
95504_2021JAN_jobAD.txt   4
95506_2021JAN_jobAD.txt   12
95507_2021JAN_jobAD.txt   9
95510_2021JAN_jobAD.txt   14
95511_2021JAN_jobAD.txt   11
95515_2021JAN_jobAD.txt   14
95518_2021JAN_jobAD.txt   6
95520_2021JAN_jobAD.txt   61
95523_2021JAN_jobAD.txt   14
95526_2021JAN_jobAD.txt   6
95527_2021JAN_jobAD.txt   7
95528_2021JAN_jobAD.txt   2
95537_2021JAN_jobAD.txt   1
95542_2021JAN_jobAD.txt   5
95544_2021JAN_jobAD.txt   2
95545_2021JAN_jobAD.txt   2
95547_2021JAN_jobAD.txt   29
95548_2021JAN_jobAD.txt   1
95555_2021JAN_jobAD.txt   9
95556_2

96322_2021JAN_jobAD.txt   8
96325_2021JAN_jobAD.txt   1
96326_2021JAN_jobAD.txt   11
96328_2021JAN_jobAD.txt   14
96330_2021JAN_jobAD.txt   22
96333_2021JAN_jobAD.txt   3
96341_2021JAN_jobAD.txt   15
96343_2021JAN_jobAD.txt   2
96350_2021JAN_jobAD.txt   17
96351_2021JAN_jobAD.txt   3
96354_2021JAN_jobAD.txt   4
96355_2021JAN_jobAD.txt   12
96356_2021JAN_jobAD.txt   11
96358_2021JAN_jobAD.txt   14
96361_2021JAN_jobAD.txt   6
96373_2021JAN_jobAD.txt   5
96374_2021JAN_jobAD.txt   13
96375_2021JAN_jobAD.txt   7
96377_2021JAN_jobAD.txt   15
96379_2021JAN_jobAD.txt   5
96381_2021JAN_jobAD.txt   2
96385_2021JAN_jobAD.txt   17
96388_2021JAN_jobAD.txt   20
96392_2021JAN_jobAD.txt   13
96399_2021JAN_jobAD.txt   3
96400_2021JAN_jobAD.txt   9
96403_2021JAN_jobAD.txt   8
96404_2021JAN_jobAD.txt   2
96406_2021JAN_jobAD.txt   3
96407_2021JAN_jobAD.txt   3
96409_2021JAN_jobAD.txt   11
96415_2021JAN_jobAD.txt   12
96416_2021JAN_jobAD.txt   7
96417_2021JAN_jobAD.txt   3
96431_2021JAN_jobAD.txt   5
96435

97172_2021JAN_jobAD.txt   4
97176_2021JAN_jobAD.txt   2
97178_2021JAN_jobAD.txt   4
97179_2021JAN_jobAD.txt   10
97186_2021JAN_jobAD.txt   10
97187_2021JAN_jobAD.txt   239
97188_2021JAN_jobAD.txt   14
97192_2021JAN_jobAD.txt   44
97197_2021JAN_jobAD.txt   15
97198_2021JAN_jobAD.txt   9
97201_2021JAN_jobAD.txt   33
97202_2021JAN_jobAD.txt   23
97203_2021JAN_jobAD.txt   3
97205_2021JAN_jobAD.txt   2
97208_2021JAN_jobAD.txt   4
97209_2021JAN_jobAD.txt   16
97210_2021JAN_jobAD.txt   3
97213_2021JAN_jobAD.txt   34
97219_2021JAN_jobAD.txt   5
97220_2021JAN_jobAD.txt   34
97221_2021JAN_jobAD.txt   6
97222_2021JAN_jobAD.txt   7
97223_2021JAN_jobAD.txt   10
97224_2021JAN_jobAD.txt   2
97225_2021JAN_jobAD.txt   2
97227_2021JAN_jobAD.txt   9
97238_2021JAN_jobAD.txt   8
97240_2021JAN_jobAD.txt   3
97246_2021JAN_jobAD.txt   18
97248_2021JAN_jobAD.txt   11
97249_2021JAN_jobAD.txt   3
97252_2021JAN_jobAD.txt   15
97255_2021JAN_jobAD.txt   3
97257_2021JAN_jobAD.txt   22
97258_2021JAN_jobAD.txt   1
972

97983_2021JAN_jobAD.txt   20
97984_2021JAN_jobAD.txt   14
97985_2021JAN_jobAD.txt   18
97991_2021JAN_jobAD.txt   2
97992_2021JAN_jobAD.txt   3
97994_2021JAN_jobAD.txt   2
97997_2021JAN_jobAD.txt   1
97999_2021JAN_jobAD.txt   36
98001_2021JAN_jobAD.txt   117
98013_2021JAN_jobAD.txt   11
98014_2021JAN_jobAD.txt   17
98016_2021JAN_jobAD.txt   2
98019_2021JAN_jobAD.txt   3
98021_2021JAN_jobAD.txt   16
98024_2021JAN_jobAD.txt   14
98026_2021JAN_jobAD.txt   3
98029_2021JAN_jobAD.txt   11
98034_2021JAN_jobAD.txt   11
98039_2021JAN_jobAD.txt   8
98041_2021JAN_jobAD.txt   3
98048_2021JAN_jobAD.txt   1
98049_2021JAN_jobAD.txt   6
98050_2021JAN_jobAD.txt   6
98054_2021JAN_jobAD.txt   3
98055_2021JAN_jobAD.txt   11
98056_2021JAN_jobAD.txt   14
98057_2021JAN_jobAD.txt   9
98060_2021JAN_jobAD.txt   6
98061_2021JAN_jobAD.txt   6
98063_2021JAN_jobAD.txt   3
98067_2021JAN_jobAD.txt   3
98069_2021JAN_jobAD.txt   11
98071_2021JAN_jobAD.txt   11
98072_2021JAN_jobAD.txt   3
98074_2021JAN_jobAD.txt   3
9807

98865_2021JAN_jobAD.txt   10
98868_2021JAN_jobAD.txt   4
98872_2021JAN_jobAD.txt   43
98875_2021JAN_jobAD.txt   4
98877_2021JAN_jobAD.txt   38
98883_2021JAN_jobAD.txt   6
98890_2021JAN_jobAD.txt   1
98892_2021JAN_jobAD.txt   2
98893_2021JAN_jobAD.txt   41
98896_2021JAN_jobAD.txt   3
98898_2021JAN_jobAD.txt   15
98899_2021JAN_jobAD.txt   10
98901_2021JAN_jobAD.txt   10
98903_2021JAN_jobAD.txt   1
98910_2021JAN_jobAD.txt   11
98912_2021JAN_jobAD.txt   11
98920_2021JAN_jobAD.txt   3
98922_2021JAN_jobAD.txt   2
98923_2021JAN_jobAD.txt   2
98928_2021JAN_jobAD.txt   33
98933_2021JAN_jobAD.txt   5
98935_2021JAN_jobAD.txt   11
98939_2021JAN_jobAD.txt   177
98940_2021JAN_jobAD.txt   20
98948_2021JAN_jobAD.txt   3
98949_2021JAN_jobAD.txt   8
98952_2021JAN_jobAD.txt   6
98955_2021JAN_jobAD.txt   2
98956_2021JAN_jobAD.txt   14
98958_2021JAN_jobAD.txt   1
98964_2021JAN_jobAD.txt   3
98967_2021JAN_jobAD.txt   14
98972_2021JAN_jobAD.txt   4
98975_2021JAN_jobAD.txt   1
98978_2021JAN_jobAD.txt   2
9897

In [ ]:
#ITJobsList = []
#for i in range(0, len(urlList)):
#    filename = "./dat/"+str(i+1)+"_2021JAN_jobAD.txt"
#    if (os.path.exists(filename)):
#        with open(filename, encoding='utf-8') as fin:
#            all_ITs = re.findall(r"(?=("+'|'.join(itList)+r"))", fin.read())
#            nITs = len(all_ITs)
#            if nITs > 0:
#                print('tech? ',str(i+1)+"_2021JAN_jobAD.txt",' ',nITs)
#                ITJobsList.append(i+1)
#            else:
#                continue
#    else:
#        continue
        
#print(ITJobsList)

In [ ]:
#USAITJobsList = []
#fout = open("./ads/2021JAN_USAITjobAD.txt","w")
#for i in range(0, len(ITJobsList)):
#    filename = "./dat/"+str(ITJobsList[i])+"_2021JAN_jobAD.txt"
#    if (os.path.exists(filename)):
#        with open(filename, encoding='utf-8') as fin:
#            all_USAs = re.findall(r"(?=("+'|'.join(statesList)+r"))", fin.read())
#            nUSAs = len(all_USAs)
#            if nUSAs > 0:
#                print('USA tech? ',str(ITJobsList[i])+"_2021JAN_jobAD.txt",' ',nUSAs)
#                fout.write(str(ITJobsList[i])+'_2021JAN_jobAD.txt'+' '+str(nUSAs)+'\n')
#                USAITJobsList.append(ITJobsList[i])
#            else:
#                continue
#    else:
#        continue
            
#fout.close()
#print(USAITJobsList)

In [19]:
fout = open("./ads/2021JAN_USAQAs.txt","w")
for i in range(0, len(USAJobsList)):
    filename = "./dat/100k_jobADs/"+str(USAJobsList[i])+"_2021JAN_jobAD.txt"
    if (os.path.exists(filename)):
        with open(filename, encoding='utf-8') as fin:
            all_QAs = re.findall(r"(?=("+'|'.join(qaList)+r"))", fin.read())
            nQAs = len(all_QAs)
            if nQAs > 0:
                print('USA QAs? ',str(USAJobsList[i])+"_2021JAN_jobAD.txt",' ',nQAs)
                fout.write(str(USAJobsList[i])+'_2021JAN_jobAD.txt'+' '+str(nQAs)+'\n')
            else:
                continue
    else:
        continue
            
fout.close()

USA QAs?  27_2021JAN_jobAD.txt   1
USA QAs?  28_2021JAN_jobAD.txt   2
USA QAs?  96_2021JAN_jobAD.txt   2
USA QAs?  129_2021JAN_jobAD.txt   1
USA QAs?  221_2021JAN_jobAD.txt   2
USA QAs?  285_2021JAN_jobAD.txt   1
USA QAs?  293_2021JAN_jobAD.txt   1
USA QAs?  312_2021JAN_jobAD.txt   1
USA QAs?  362_2021JAN_jobAD.txt   1
USA QAs?  410_2021JAN_jobAD.txt   2
USA QAs?  453_2021JAN_jobAD.txt   1
USA QAs?  539_2021JAN_jobAD.txt   3
USA QAs?  575_2021JAN_jobAD.txt   1
USA QAs?  578_2021JAN_jobAD.txt   3
USA QAs?  610_2021JAN_jobAD.txt   3
USA QAs?  625_2021JAN_jobAD.txt   11
USA QAs?  629_2021JAN_jobAD.txt   1
USA QAs?  641_2021JAN_jobAD.txt   2
USA QAs?  681_2021JAN_jobAD.txt   3
USA QAs?  683_2021JAN_jobAD.txt   1
USA QAs?  763_2021JAN_jobAD.txt   1
USA QAs?  783_2021JAN_jobAD.txt   2
USA QAs?  786_2021JAN_jobAD.txt   1
USA QAs?  798_2021JAN_jobAD.txt   1
USA QAs?  807_2021JAN_jobAD.txt   1
USA QAs?  809_2021JAN_jobAD.txt   3
USA QAs?  810_2021JAN_jobAD.txt   8
USA QAs?  811_2021JAN_jobAD.tx

USA QAs?  4261_2021JAN_jobAD.txt   2
USA QAs?  4278_2021JAN_jobAD.txt   2
USA QAs?  4313_2021JAN_jobAD.txt   1
USA QAs?  4319_2021JAN_jobAD.txt   1
USA QAs?  4337_2021JAN_jobAD.txt   1
USA QAs?  4347_2021JAN_jobAD.txt   1
USA QAs?  4356_2021JAN_jobAD.txt   2
USA QAs?  4362_2021JAN_jobAD.txt   2
USA QAs?  4377_2021JAN_jobAD.txt   1
USA QAs?  4382_2021JAN_jobAD.txt   1
USA QAs?  4402_2021JAN_jobAD.txt   1
USA QAs?  4407_2021JAN_jobAD.txt   2
USA QAs?  4423_2021JAN_jobAD.txt   1
USA QAs?  4432_2021JAN_jobAD.txt   3
USA QAs?  4445_2021JAN_jobAD.txt   2
USA QAs?  4459_2021JAN_jobAD.txt   1
USA QAs?  4502_2021JAN_jobAD.txt   1
USA QAs?  4504_2021JAN_jobAD.txt   1
USA QAs?  4533_2021JAN_jobAD.txt   1
USA QAs?  4546_2021JAN_jobAD.txt   1
USA QAs?  4558_2021JAN_jobAD.txt   2
USA QAs?  4579_2021JAN_jobAD.txt   1
USA QAs?  4597_2021JAN_jobAD.txt   2
USA QAs?  4602_2021JAN_jobAD.txt   2
USA QAs?  4603_2021JAN_jobAD.txt   1
USA QAs?  4616_2021JAN_jobAD.txt   3
USA QAs?  4660_2021JAN_jobAD.txt   1
U

USA QAs?  8280_2021JAN_jobAD.txt   2
USA QAs?  8284_2021JAN_jobAD.txt   1
USA QAs?  8301_2021JAN_jobAD.txt   2
USA QAs?  8355_2021JAN_jobAD.txt   3
USA QAs?  8362_2021JAN_jobAD.txt   4
USA QAs?  8366_2021JAN_jobAD.txt   1
USA QAs?  8379_2021JAN_jobAD.txt   1
USA QAs?  8416_2021JAN_jobAD.txt   1
USA QAs?  8424_2021JAN_jobAD.txt   2
USA QAs?  8430_2021JAN_jobAD.txt   1
USA QAs?  8435_2021JAN_jobAD.txt   1
USA QAs?  8455_2021JAN_jobAD.txt   1
USA QAs?  8494_2021JAN_jobAD.txt   14
USA QAs?  8510_2021JAN_jobAD.txt   7
USA QAs?  8511_2021JAN_jobAD.txt   2
USA QAs?  8516_2021JAN_jobAD.txt   1
USA QAs?  8528_2021JAN_jobAD.txt   3
USA QAs?  8534_2021JAN_jobAD.txt   2
USA QAs?  8561_2021JAN_jobAD.txt   1
USA QAs?  8582_2021JAN_jobAD.txt   1
USA QAs?  8611_2021JAN_jobAD.txt   1
USA QAs?  8626_2021JAN_jobAD.txt   1
USA QAs?  8634_2021JAN_jobAD.txt   1
USA QAs?  8678_2021JAN_jobAD.txt   1
USA QAs?  8687_2021JAN_jobAD.txt   1
USA QAs?  8692_2021JAN_jobAD.txt   3
USA QAs?  8708_2021JAN_jobAD.txt   1


USA QAs?  12551_2021JAN_jobAD.txt   1
USA QAs?  12555_2021JAN_jobAD.txt   1
USA QAs?  12568_2021JAN_jobAD.txt   1
USA QAs?  12629_2021JAN_jobAD.txt   1
USA QAs?  12632_2021JAN_jobAD.txt   1
USA QAs?  12641_2021JAN_jobAD.txt   2
USA QAs?  12692_2021JAN_jobAD.txt   12
USA QAs?  12717_2021JAN_jobAD.txt   14
USA QAs?  12720_2021JAN_jobAD.txt   1
USA QAs?  12721_2021JAN_jobAD.txt   1
USA QAs?  12732_2021JAN_jobAD.txt   1
USA QAs?  12755_2021JAN_jobAD.txt   1
USA QAs?  12760_2021JAN_jobAD.txt   2
USA QAs?  12768_2021JAN_jobAD.txt   1
USA QAs?  12807_2021JAN_jobAD.txt   1
USA QAs?  12812_2021JAN_jobAD.txt   1
USA QAs?  12849_2021JAN_jobAD.txt   1
USA QAs?  12887_2021JAN_jobAD.txt   1
USA QAs?  12894_2021JAN_jobAD.txt   2
USA QAs?  12902_2021JAN_jobAD.txt   2
USA QAs?  12904_2021JAN_jobAD.txt   1
USA QAs?  13000_2021JAN_jobAD.txt   1
USA QAs?  13005_2021JAN_jobAD.txt   14
USA QAs?  13023_2021JAN_jobAD.txt   3
USA QAs?  13036_2021JAN_jobAD.txt   2
USA QAs?  13059_2021JAN_jobAD.txt   2
USA QAs? 

USA QAs?  16256_2021JAN_jobAD.txt   1
USA QAs?  16257_2021JAN_jobAD.txt   1
USA QAs?  16311_2021JAN_jobAD.txt   1
USA QAs?  16327_2021JAN_jobAD.txt   3
USA QAs?  16350_2021JAN_jobAD.txt   2
USA QAs?  16366_2021JAN_jobAD.txt   1
USA QAs?  16386_2021JAN_jobAD.txt   14
USA QAs?  16418_2021JAN_jobAD.txt   1
USA QAs?  16420_2021JAN_jobAD.txt   1
USA QAs?  16444_2021JAN_jobAD.txt   1
USA QAs?  16448_2021JAN_jobAD.txt   1
USA QAs?  16454_2021JAN_jobAD.txt   1
USA QAs?  16470_2021JAN_jobAD.txt   1
USA QAs?  16473_2021JAN_jobAD.txt   1
USA QAs?  16489_2021JAN_jobAD.txt   2
USA QAs?  16491_2021JAN_jobAD.txt   1
USA QAs?  16504_2021JAN_jobAD.txt   2
USA QAs?  16510_2021JAN_jobAD.txt   4
USA QAs?  16512_2021JAN_jobAD.txt   1
USA QAs?  16538_2021JAN_jobAD.txt   3
USA QAs?  16556_2021JAN_jobAD.txt   3
USA QAs?  16569_2021JAN_jobAD.txt   3
USA QAs?  16573_2021JAN_jobAD.txt   2
USA QAs?  16600_2021JAN_jobAD.txt   2
USA QAs?  16612_2021JAN_jobAD.txt   1
USA QAs?  16615_2021JAN_jobAD.txt   2
USA QAs?  1

...

USA QAs?  83068_2021JAN_jobAD.txt   2
USA QAs?  83094_2021JAN_jobAD.txt   1
USA QAs?  83103_2021JAN_jobAD.txt   2
USA QAs?  83178_2021JAN_jobAD.txt   1
USA QAs?  83181_2021JAN_jobAD.txt   1
USA QAs?  83209_2021JAN_jobAD.txt   19
USA QAs?  83219_2021JAN_jobAD.txt   1
USA QAs?  83221_2021JAN_jobAD.txt   2
USA QAs?  83224_2021JAN_jobAD.txt   1
USA QAs?  83228_2021JAN_jobAD.txt   1
USA QAs?  83254_2021JAN_jobAD.txt   4
USA QAs?  83291_2021JAN_jobAD.txt   1
USA QAs?  83350_2021JAN_jobAD.txt   1
USA QAs?  83390_2021JAN_jobAD.txt   2
USA QAs?  83392_2021JAN_jobAD.txt   1
USA QAs?  83398_2021JAN_jobAD.txt   1
USA QAs?  83456_2021JAN_jobAD.txt   1
USA QAs?  83489_2021JAN_jobAD.txt   1
USA QAs?  83496_2021JAN_jobAD.txt   3
USA QAs?  83561_2021JAN_jobAD.txt   2
USA QAs?  83567_2021JAN_jobAD.txt   1
USA QAs?  83578_2021JAN_jobAD.txt   2
USA QAs?  83580_2021JAN_jobAD.txt   1
USA QAs?  83588_2021JAN_jobAD.txt   2
USA QAs?  83618_2021JAN_jobAD.txt   3
USA QAs?  83633_2021JAN_jobAD.txt   2
USA QAs?  8

USA QAs?  86514_2021JAN_jobAD.txt   14
USA QAs?  86548_2021JAN_jobAD.txt   1
USA QAs?  86575_2021JAN_jobAD.txt   2
USA QAs?  86580_2021JAN_jobAD.txt   1
USA QAs?  86609_2021JAN_jobAD.txt   1
USA QAs?  86630_2021JAN_jobAD.txt   1
USA QAs?  86635_2021JAN_jobAD.txt   1
USA QAs?  86676_2021JAN_jobAD.txt   1
USA QAs?  86687_2021JAN_jobAD.txt   3
USA QAs?  86701_2021JAN_jobAD.txt   1
USA QAs?  86703_2021JAN_jobAD.txt   1
USA QAs?  86707_2021JAN_jobAD.txt   2
USA QAs?  86726_2021JAN_jobAD.txt   2
USA QAs?  86727_2021JAN_jobAD.txt   2
USA QAs?  86749_2021JAN_jobAD.txt   2
USA QAs?  86770_2021JAN_jobAD.txt   2
USA QAs?  86793_2021JAN_jobAD.txt   1
USA QAs?  86798_2021JAN_jobAD.txt   1
USA QAs?  86838_2021JAN_jobAD.txt   3
USA QAs?  86857_2021JAN_jobAD.txt   2
USA QAs?  86888_2021JAN_jobAD.txt   1
USA QAs?  86895_2021JAN_jobAD.txt   1
USA QAs?  86907_2021JAN_jobAD.txt   1
USA QAs?  86908_2021JAN_jobAD.txt   1
USA QAs?  86917_2021JAN_jobAD.txt   2
USA QAs?  86946_2021JAN_jobAD.txt   2
USA QAs?  8

USA QAs?  90475_2021JAN_jobAD.txt   1
USA QAs?  90494_2021JAN_jobAD.txt   1
USA QAs?  90524_2021JAN_jobAD.txt   1
USA QAs?  90530_2021JAN_jobAD.txt   1
USA QAs?  90542_2021JAN_jobAD.txt   14
USA QAs?  90623_2021JAN_jobAD.txt   1
USA QAs?  90628_2021JAN_jobAD.txt   4
USA QAs?  90633_2021JAN_jobAD.txt   1
USA QAs?  90681_2021JAN_jobAD.txt   2
USA QAs?  90685_2021JAN_jobAD.txt   1
USA QAs?  90700_2021JAN_jobAD.txt   1
USA QAs?  90729_2021JAN_jobAD.txt   1
USA QAs?  90734_2021JAN_jobAD.txt   1
USA QAs?  90757_2021JAN_jobAD.txt   3
USA QAs?  90788_2021JAN_jobAD.txt   2
USA QAs?  90812_2021JAN_jobAD.txt   1
USA QAs?  90874_2021JAN_jobAD.txt   1
USA QAs?  90876_2021JAN_jobAD.txt   1
USA QAs?  90897_2021JAN_jobAD.txt   3
USA QAs?  90911_2021JAN_jobAD.txt   1
USA QAs?  90938_2021JAN_jobAD.txt   2
USA QAs?  90945_2021JAN_jobAD.txt   1
USA QAs?  90956_2021JAN_jobAD.txt   1
USA QAs?  90957_2021JAN_jobAD.txt   1
USA QAs?  90972_2021JAN_jobAD.txt   1
USA QAs?  90986_2021JAN_jobAD.txt   1
USA QAs?  9

USA QAs?  94241_2021JAN_jobAD.txt   3
USA QAs?  94248_2021JAN_jobAD.txt   15
USA QAs?  94254_2021JAN_jobAD.txt   3
USA QAs?  94256_2021JAN_jobAD.txt   1
USA QAs?  94278_2021JAN_jobAD.txt   1
USA QAs?  94283_2021JAN_jobAD.txt   1
USA QAs?  94312_2021JAN_jobAD.txt   14
USA QAs?  94322_2021JAN_jobAD.txt   1
USA QAs?  94324_2021JAN_jobAD.txt   7
USA QAs?  94357_2021JAN_jobAD.txt   1
USA QAs?  94391_2021JAN_jobAD.txt   2
USA QAs?  94408_2021JAN_jobAD.txt   1
USA QAs?  94418_2021JAN_jobAD.txt   3
USA QAs?  94458_2021JAN_jobAD.txt   1
USA QAs?  94483_2021JAN_jobAD.txt   1
USA QAs?  94502_2021JAN_jobAD.txt   1
USA QAs?  94518_2021JAN_jobAD.txt   1
USA QAs?  94523_2021JAN_jobAD.txt   1
USA QAs?  94539_2021JAN_jobAD.txt   1
USA QAs?  94547_2021JAN_jobAD.txt   1
USA QAs?  94565_2021JAN_jobAD.txt   1
USA QAs?  94585_2021JAN_jobAD.txt   1
USA QAs?  94684_2021JAN_jobAD.txt   1
USA QAs?  94691_2021JAN_jobAD.txt   1
USA QAs?  94701_2021JAN_jobAD.txt   1
USA QAs?  94732_2021JAN_jobAD.txt   1
USA QAs?  

USA QAs?  97913_2021JAN_jobAD.txt   4
USA QAs?  97928_2021JAN_jobAD.txt   1
USA QAs?  97932_2021JAN_jobAD.txt   1
USA QAs?  97934_2021JAN_jobAD.txt   3
USA QAs?  97935_2021JAN_jobAD.txt   1
USA QAs?  97957_2021JAN_jobAD.txt   2
USA QAs?  97962_2021JAN_jobAD.txt   14
USA QAs?  97978_2021JAN_jobAD.txt   1
USA QAs?  97983_2021JAN_jobAD.txt   1
USA QAs?  97997_2021JAN_jobAD.txt   1
USA QAs?  98001_2021JAN_jobAD.txt   1
USA QAs?  98013_2021JAN_jobAD.txt   1
USA QAs?  98014_2021JAN_jobAD.txt   1
USA QAs?  98016_2021JAN_jobAD.txt   1
USA QAs?  98050_2021JAN_jobAD.txt   6
USA QAs?  98060_2021JAN_jobAD.txt   1
USA QAs?  98131_2021JAN_jobAD.txt   5
USA QAs?  98153_2021JAN_jobAD.txt   1
USA QAs?  98155_2021JAN_jobAD.txt   1
USA QAs?  98160_2021JAN_jobAD.txt   1
USA QAs?  98163_2021JAN_jobAD.txt   1
USA QAs?  98165_2021JAN_jobAD.txt   1
USA QAs?  98179_2021JAN_jobAD.txt   1
USA QAs?  98185_2021JAN_jobAD.txt   1
USA QAs?  98190_2021JAN_jobAD.txt   3
USA QAs?  98214_2021JAN_jobAD.txt   1
USA QAs?  9